In [ ]:
import cv2
import numpy as np

In [ ]:
for i in range(1,100):
    # Read image
    img = cv2.imread('Leafs/leaf (' + str(i) + ').jpg')
    hh, ww = img.shape[:2]

    # threshold on white
    # Define lower and uppper limits
    lower = np.array([190, 190, 190])
    upper = np.array([255, 255, 255])

    # Create mask to only select black
    thresh = cv2.inRange(img, lower, upper)

    # apply morphology
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20,20))
    morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    # invert morp image
    mask = 255 - morph

    # apply mask to image
    result = cv2.bitwise_and(img, img, mask=mask)


    # save results
    cv2.imwrite('leaf_result(' + str(i) + '.jpg', result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [14]:
%%time
import numpy as np
import cv2
import os

def process_image(image: np.ndarray) -> np.ndarray:
    lower = np.array([128, 64, 128])
    upper = np.array([255, 255, 255])
    thresh = cv2.inRange(image, lower, upper)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10))
    morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    mask = 255 - morph
    result = cv2.bitwise_and(image, image, mask=mask)
    return result

def process_folder(folder: str, target_folder: str):
    files = os.listdir(folder)
    for filename in files[:100]:
        image = cv2.imread(os.path.join(folder, filename))
        image = cv2.resize(image, (1024, 1024))
        backed = process_image(image)
        cv2.imwrite(os.path.join(target_folder, filename), backed)

process_folder("RiceLeafs/train/Healthy", "test_back")

CPU times: user 9.34 s, sys: 370 ms, total: 9.71 s
Wall time: 8.94 s


In [4]:
def find_intersection(k1: float, b1: float, k2: float, b2: float, x1: int, size: int) -> tuple[int, int]:
    y1 = k1 * x1 + b1
    x_r = (y1 + x1 / k1 - b2) / (k2 + 1 / k1)
    y_r = x_r * k2 + b2
    print(k1, b1, k2, b2, x1, x_r, y_r)
    if x_r < 0:
        #return int(0), int(b2)
        #return find_intersection(k1, b1, k2, b2, x1 - x_r, size)
        return find_intersection(k2, b2, k1, b1, 1, size)
    elif y_r < 0:
        #return int(-b2 / k2), int(0)
        #return find_intersection(k1, b1, k2, b2, x1 - y_r / k1, size)
        res = find_intersection(k2, b2, k1, b1, int((1 - b2) / k2), size)
        return find_intersection(k1, b1, k2, b2, res[0], size)
    elif x_r >= size:
        #return int(size - 1), int((size - 1) * k2 + b2)
        #return find_intersection(k1, b1, k2, b2, x1 + (size - x_r), size)
        return find_intersection(k2, b2, k1, b1, int(size - 1), size)
    elif y_r >= size:
        #return int((size - 1) - b2 / k2), int((size - 1))
        #return find_intersection(k1, b1, k2, b2, x1 + (size - y_r) / k1, size)
        return find_intersection(k2, b2, k1, b1, int(((size - 1) - b2) / k2), size)
    return int(x_r), int(y_r)

print(find_intersection(1, -1, 1, -2,  1, 10))
print(find_intersection(1, 1, 1, 2,  0, 10))
print(find_intersection(1, 1, 1, -2,  0, 10))

1 -1 1 -2 1 1.5 -0.5
1 -2 1 -1 3 2.5 1.5
1 -1 1 -2 2 2.5 0.5
(2, 0)
1 1 1 2 0 -0.5 1.5
1 2 1 1 1 1.5 2.5
(1, 2)
1 1 1 -2 0 1.5 -0.5
1 -2 1 1 3 1.5 2.5
1 1 1 -2 1 2.5 0.5
(2, 0)
